In [2]:
#Benötigte Packages
import os
import requests
import xml.etree.ElementTree as ET
import pandas as pd

In [3]:
# Grundeinstellungen

# bestandteile der URL
## alle Zahlen in Grad

lons = [13.6]  # linker Rand
lats = [51.0]  # unterer Rand
zellweite_Lon = 0.1  # pro 0.1 ist die Zelle ca 7km breiter/schmaler
zellweite_Lat = 0.1  # pro 0.1 ist die Zelle ca 11km höher/niedriger
url1 = 'https://api.openstreetmap.org/api/0.6/trackpoints?bbox='  # bestandteile URL
url2 = '&page='  # bestandteile URL

In [ ]:
# BBOX und URL
# Schleife für das erstellen der Dateien und hinzufügen der Punkte zu einem df
for lon in lons:
    for lat in lats:
        bbox = f"{lon},{lat},{lon + zellweite_Lon},{lat + zellweite_Lat}"  # erstellen der BBOX
        page = 0
        os.makedirs("osmtracks", exist_ok=True)  # Ordner erstellen um die .gpx zu speichern
        while True:
            url = f"{url1}{bbox}{url2}{page}"
            response = requests.get(url)
            filestring = f"osmtracks/tracks_{page}.gpx"  # Dateipfad und Name
            with open(filestring, "wb") as file:
                file.write(response.content)
            
            # datei direkt in xml umwandeln
            new_files = filestring.replace(".gpx", ".xml")
            os.rename(filestring, new_files)
    
            page += 1  # Page um eins erhöhen
            
            size = os.path.getsize(new_files)  # Dateigröße für Abbruchbedingung
            # Abbruchbedingung sehr wichtig, sonst wiederholt sich die letzte Page unendlich oft, bessere Alternative als
            # Dateigröße finden
            if size < 500000:
                break


In [11]:
# Benötigte Dataframes usw.
tracks_df = pd.DataFrame(columns=["lat", "lon", "time", "seg_id", "pt_id", "name"])
track_df = pd.DataFrame(columns=["lat", "lon", "time", "seg_id", "pt_id", "name"])

In [10]:
files = []

for file in os.listdir("/Forschungsprojekt/osmtracks"):
    if file.endswith("tracks_0.xml"):
        files.append(file)

for file in files:
    tree = ET.parse("/Forschungsprojekt/osmtracks/" + file)
    root = tree.getroot()

    for trk in root.findall(".//{http://www.topografix.com/GPX/1/0}trk"):
        seg_counter = 0
        for trkseg in trk.findall(".//{http://www.topografix.com/GPX/1/0}trkseg"):
            seg_counter += 1
            seg_id = seg_counter
            pt_id = 0
            for trkpt in trkseg.findall(".//{http://www.topografix.com/GPX/1/0}trkpt"):
                lat = trkpt.get("lat")
                lon = trkpt.get("lon")
                time = trkpt.find(".//{http://www.topografix.com/GPX/1/0}time").text if trkpt.find(".//{http://www.topografix.com/GPX/1/0}time") is not None else "NA"
                name = trkpt.find(".//{http://www.topografix.com/GPX/1/0}name").text if trkpt.find(".//{http://www.topografix.com/GPX/1/0}name") is not None else "NoName"
                pt_id += 1
                new_row = [lat, lon, time, seg_id, pt_id, name]
                track_df.loc[len(track_df)] = new_row

    tracks_df = pd.concat([tracks_df, track_df], ignore_index=True)
    track_df = pd.DataFrame(columns=["lat", "lon", "time", "seg_id", "pt_id", "name"])


KeyboardInterrupt: 